# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [91]:
# import libraries
from pyspark.sql import SparkSession
import pyspark.sql.functions as psqf
import pyspark.sql.types as psqt
from pyspark.ml.feature import VectorAssembler, StandardScaler\

from pyspark.mllib.tree import RandomForest, RandomForestModel, LabeledPoint
from pyspark.mllib.util import MLUtils

import datetime
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline


https://github.com/klrpdx/sparkify

2.https://github.com/fxzero/Sparkify-Project

More Options 3.https://github.com/linpingyu/Sparkify

4.https://github.com/sanjeevai/sparkify-capstone

5.https://github.com/gharesh/Sparkify-Project

6.https://medium.com/@sandeep.nie1/sparkify-big-data-udacity-capstone-project-52be7b3c0414

In [2]:
# create a Spark session
spark = SparkSession.builder.appName("customer-churn data pipeline").getOrCreate()

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [3]:
data_path = "mini_sparkify_event_data.json"
user_event = spark.read.json(data_path)

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

**Preliminary data analysis**

In [4]:
user_event.show(2)

+----------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+---------+------+-------------+--------------------+------+
|          artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page| registration|sessionId|     song|status|           ts|           userAgent|userId|
+----------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+---------+------+-------------+--------------------+------+
|  Martha Tilston|Logged In|    Colin|     M|           50| Freeman|277.89016| paid|     Bakersfield, CA|   PUT|NextSong|1538173362000|       29|Rockpools|   200|1538352117000|Mozilla/5.0 (Wind...|    30|
|Five Iron Frenzy|Logged In|    Micah|     M|           79|    Long|236.09424| free|Boston-Cambridge-...|   PUT|NextSong|1538331630000|        8|   Canada|   200|1538352180000|"Moz

In [5]:
print(user_event.count(), len(user_event.columns))

286500 18


In [6]:
user_event.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [7]:
#count null values for each column
df_nulls = user_event.select([psqf.count(psqf.when(psqf.isnull(c), c)).alias(c) for c in user_event.columns])

In [8]:
user_event.filter(psqf.isnull(psqf.col('artist'))).count()

58392

In [9]:
# let's make sure page column doesn't have nulls ( we'll define churn on 'page' column)
df_nulls.select('page').show()

+----+
|page|
+----+
|   0|
+----+



In [10]:
user_event.select('level').distinct().show()

+-----+
|level|
+-----+
| free|
| paid|
+-----+



In [11]:
level_counts= user_event.groupby('level').agg({'level':'count'}).withColumnRenamed("count(level)", "level_count")
level_counts.show()

+-----+-----------+
|level|level_count|
+-----+-----------+
| free|      58338|
| paid|     228162|
+-----+-----------+



In [12]:
user_event.select('page').distinct().show()

+--------------------+
|                page|
+--------------------+
|              Cancel|
|    Submit Downgrade|
|         Thumbs Down|
|                Home|
|           Downgrade|
|         Roll Advert|
|              Logout|
|       Save Settings|
|Cancellation Conf...|
|               About|
| Submit Registration|
|            Settings|
|               Login|
|            Register|
|     Add to Playlist|
|          Add Friend|
|            NextSong|
|           Thumbs Up|
|                Help|
|             Upgrade|
+--------------------+
only showing top 20 rows



In [13]:
page_counts= user_event.groupby('page').agg({'page':'count'}).withColumnRenamed("count(page)", "page_count")
page_counts.show()

+--------------------+----------+
|                page|page_count|
+--------------------+----------+
|              Cancel|        52|
|    Submit Downgrade|        63|
|         Thumbs Down|      2546|
|                Home|     14457|
|           Downgrade|      2055|
|         Roll Advert|      3933|
|              Logout|      3226|
|       Save Settings|       310|
|Cancellation Conf...|        52|
|               About|       924|
| Submit Registration|         5|
|            Settings|      1514|
|               Login|      3241|
|            Register|        18|
|     Add to Playlist|      6526|
|          Add Friend|      4277|
|            NextSong|    228108|
|           Thumbs Up|     12551|
|                Help|      1726|
|             Upgrade|       499|
+--------------------+----------+
only showing top 20 rows



Let's look at **Cancel** and **Cancellation Confirmation** samples in the dataset

In [14]:
cancel_events = user_event.filter(psqf.col('page').isin(['Cancel','Cancellation Confirmation'])).select(['userID','page', 'firstName', 'lastName','ts', 'auth'])
cancel_events.show(5, False)

+------+-------------------------+---------+--------+-------------+---------+
|userID|page                     |firstName|lastName|ts           |auth     |
+------+-------------------------+---------+--------+-------------+---------+
|18    |Cancel                   |Adriel   |Mendoza |1538943740000|Logged In|
|18    |Cancellation Confirmation|Adriel   |Mendoza |1538943990000|Cancelled|
|32    |Cancel                   |Diego    |Mckee   |1539033031000|Logged In|
|32    |Cancellation Confirmation|Diego    |Mckee   |1539033046000|Cancelled|
|125   |Cancel                   |Mason    |Hart    |1539318918000|Logged In|
+------+-------------------------+---------+--------+-------------+---------+
only showing top 5 rows



Now let's analyse the downgrade page a bit by looking at how many of the customers who have downgraded have also cancelled thier subscriptions. The registrationId uniquely identifies each customer.

In [15]:
cancel_reg_ids  = [vv['userID'] for vv in cancel_events.select('userID').collect()]
print(len(cancel_reg_ids), len(set(cancel_reg_ids)))

104 52


In [16]:
#customer who downgraded
downgrade_events = user_event.filter(psqf.col('page').isin(['Downgrade'])).select(['userID','page', 'firstName', 'lastName','ts', 'auth'])
downgrade_events.show(5, False)

+------+---------+---------+--------+-------------+---------+
|userID|page     |firstName|lastName|ts           |auth     |
+------+---------+---------+--------+-------------+---------+
|54    |Downgrade|Alexi    |Warren  |1538354749000|Logged In|
|95    |Downgrade|Faigy    |Howe    |1538373286000|Logged In|
|95    |Downgrade|Faigy    |Howe    |1538392612000|Logged In|
|131   |Downgrade|Kael     |Baker   |1538393618000|Logged In|
|95    |Downgrade|Faigy    |Howe    |1538393664000|Logged In|
+------+---------+---------+--------+-------------+---------+
only showing top 5 rows



In [17]:
downgrade_reg_ids = [vv['userID'] for vv in downgrade_events.select('userID').collect()]
print(len(downgrade_reg_ids), len(set(downgrade_reg_ids)))

2055 154


In [18]:
# Now let's see which of those who downgraded also cancel thier subscription
down_cancel = set(cancel_reg_ids).intersection((set(downgrade_reg_ids)))
print('{0:.2f}% of customers who downgraded have also cancelled their subscriptions'.format(
    100*(len(down_cancel))/len(set(downgrade_reg_ids))))

22.73% of customers who downgraded have also cancelled their subscriptions


In [19]:
list(down_cancel)[0]

'18'

In [20]:
user_event.filter((psqf.col('userID') == list(down_cancel)[0]) &
                  (psqf.col('page').isin(['Downgrade','Cancel']))).select(['userID','page', 'firstName','ts']).show()

+------+---------+---------+-------------+
|userID|     page|firstName|           ts|
+------+---------+---------+-------------+
|    18|Downgrade|   Adriel|1538749555000|
|    18|Downgrade|   Adriel|1538935253000|
|    18|Downgrade|   Adriel|1538943739000|
|    18|   Cancel|   Adriel|1538943740000|
+------+---------+---------+-------------+



Inorder to understand the sequence of events, let's convert the timestamp into datatime format.

In [21]:
ts_udf = psqf.udf(lambda x: datetime.datetime.utcfromtimestamp(x/1000.0).strftime('%Y-%m-%d %H:%M:%S'))

In [22]:
user_event = user_event.select('*').withColumn('ts_datetime', ts_udf(psqf.col('ts')))

In [23]:
user_event.filter((psqf.col('userID') == list(down_cancel)[1]) &
                  (psqf.col('page').isin(['Downgrade','Cancel', 'Cancellation Confirmation']))).select(['userID','page', 'firstName','ts', 'ts_datetime']).show(20, False)

+------+-------------------------+---------+-------------+-------------------+
|userID|page                     |firstName|ts           |ts_datetime        |
+------+-------------------------+---------+-------------+-------------------+
|100023|Downgrade                |Sawyer   |1538785799000|2018-10-06 00:29:59|
|100023|Downgrade                |Sawyer   |1538963916000|2018-10-08 01:58:36|
|100023|Downgrade                |Sawyer   |1539010201000|2018-10-08 14:50:01|
|100023|Downgrade                |Sawyer   |1539011530000|2018-10-08 15:12:10|
|100023|Downgrade                |Sawyer   |1539024088000|2018-10-08 18:41:28|
|100023|Downgrade                |Sawyer   |1539040537000|2018-10-08 23:15:37|
|100023|Downgrade                |Sawyer   |1539299163000|2018-10-11 23:06:03|
|100023|Downgrade                |Sawyer   |1539303978000|2018-10-12 00:26:18|
|100023|Downgrade                |Sawyer   |1539475171000|2018-10-13 23:59:31|
|100023|Cancel                   |Sawyer   |15394751

Loooking at just on customer, we can see that he was the downgrade page and cancelled after 1 hour.You are welcome to see some of the other users to see this is the case. Here I think the number of Downgrades a user completed could be an indication that user will terminate subscription.

**Define Churn**

The above analysis shows that **Cancel** is followed by **cancellation confirmation page** and both indicate user is canceling thier subscreption unless they change thier mind once they are on the **cancel** page which seems unlikely event at least for this dataset. So we will use both of these events to define Churn. We have also seen that about 23% of the customers who downgraded have also cancelled thier subscription. However, we'll exclude those who downgraded but didn't cancel yet for this analysis even thought they seem candidates who will eventually cancel their subscription.

In [24]:
churn_event = user_event.groupby('userId').agg(psqf.collect_list('page').alias('pages'))
# define 1 as churned, 0 otherwise
churn_f = psqf.udf(lambda x: 1 if 'Cancel' in set(x) else 0)
churn_event = churn_event.withColumn("churned", churn_f(churn_event.pages)).drop('pages')

In [27]:
churn_event.show()

+------+-------+
|userId|churned|
+------+-------+
|100010|      0|
|200002|      0|
|   125|      1|
|   124|      0|
|    51|      1|
|     7|      0|
|    15|      0|
|    54|      1|
|   155|      0|
|100014|      1|
|   132|      0|
|   154|      0|
|   101|      1|
|    11|      0|
|   138|      0|
|300017|      0|
|100021|      1|
|    29|      1|
|    69|      0|
|   112|      0|
+------+-------+
only showing top 20 rows



In [56]:
# remove features that are not useful for our analysis
clean_df = user_event.select('artist','auth','firstName','gender','lastName','length','level','location','page','song','ts','userId')

In [57]:
labeled_df  = churn_event.join(clean_df, 'userId')

In [58]:
labeled_df.show(5)

+------+-------+--------------------+---------+---------+------+---------+---------+-----+--------------------+---------+--------------------+-------------+
|userId|churned|              artist|     auth|firstName|gender| lastName|   length|level|            location|     page|                song|           ts|
+------+-------+--------------------+---------+---------+------+---------+---------+-----+--------------------+---------+--------------------+-------------+
|100010|      0|Sleeping With Sirens|Logged In| Darianna|     F|Carpenter|202.97098| free|Bridgeport-Stamfo...| NextSong|Captain Tyin Knot...|1539003534000|
|100010|      0|Francesca Battist...|Logged In| Darianna|     F|Carpenter|196.54485| free|Bridgeport-Stamfo...| NextSong|Beautiful_ Beauti...|1539003736000|
|100010|      0|              Brutha|Logged In| Darianna|     F|Carpenter|263.13098| free|Bridgeport-Stamfo...| NextSong|          She's Gone|1539003932000|
|100010|      0|                null|Logged In| Darianna| 

In [59]:
labeled_df.filter(psqf.col('page') =="Cancel").show(5)

+------+-------+------+---------+---------+------+--------+------+-----+--------------------+------+----+-------------+
|userId|churned|artist|     auth|firstName|gender|lastName|length|level|            location|  page|song|           ts|
+------+-------+------+---------+---------+------+--------+------+-----+--------------------+------+----+-------------+
|   125|      1|  null|Logged In|    Mason|     M|    Hart|  null| free|  Corpus Christi, TX|Cancel|null|1539318918000|
|    51|      1|  null|Logged In|    Ethan|     M| Johnson|  null| paid|Lexington-Fayette...|Cancel|null|1539761830000|
|    54|      1|  null|Logged In|    Alexi|     F|  Warren|  null| paid|Spokane-Spokane V...|Cancel|null|1542051577000|
|100014|      1|  null|Logged In|  Rodrigo|     M|   Carey|  null| paid|New York-Newark-J...|Cancel|null|1542740642000|
|   101|      1|  null|Logged In|     Alex|     M|   Hogan|  null| paid|Denver-Aurora-Lak...|Cancel|null|1539728810000|
+------+-------+------+---------+-------

In [60]:
labeled_df.groupby("Churned").agg({'churned': 'count'}).show()

+-------+--------------+
|Churned|count(churned)|
+-------+--------------+
|      0|        241636|
|      1|         44864|
+-------+--------------+



### Which of those features can be useful to define churn?
- ts: we can get information like total number of hours a customer played songs
- song: we can get the number of songs a customer played
- from **page** column we can find the number of ThumbsUp, ThumbsDown, and Downgrades

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

### Number of songs each user played

In [61]:
songsplayed = labeled_df.where(psqf.col('song')!='null').groupby("userId").agg(psqf.count(psqf.col('song')).alias('SongsPlayed')).orderBy('userId')
songsplayed.show(5)

+------+-----------+
|userId|SongsPlayed|
+------+-----------+
|    10|        673|
|   100|       2682|
|100001|        133|
|100002|        195|
|100003|         51|
+------+-----------+
only showing top 5 rows



### Number of distinct hour counts  a user logged in the system

In [62]:
hours_udf = psqf.udf(lambda x: datetime.datetime.utcfromtimestamp(x/1000.0).strftime('%Y-%m-%d-%H'))
hours_df  = labeled_df.select('userId', 'ts').withColumn('hour', hours_udf(psqf.col('ts')))
hours_df.show(5)

+------+-------------+-------------+
|userId|           ts|         hour|
+------+-------------+-------------+
|100010|1539003534000|2018-10-08-12|
|100010|1539003736000|2018-10-08-13|
|100010|1539003932000|2018-10-08-13|
|100010|1539003933000|2018-10-08-13|
|100010|1539004195000|2018-10-08-13|
+------+-------------+-------------+
only showing top 5 rows



In [65]:
hour_count_df = hours_df.where(psqf.col('userId')!='null').groupby('userId').agg((psqf.countDistinct(psqf.col('hour'))).alias("HourCount")).orderBy('userId')
hour_count_df.show(5)

+------+---------+
|userId|HourCount|
+------+---------+
|      |     1306|
|    10|       51|
|   100|      218|
|100001|       14|
|100002|       18|
+------+---------+
only showing top 5 rows



### Thumbs Up and Thumbs Down counts:
A user having a lot of thumbs down could be an indication of the users disastisfaction with song recommendations from Sparkify while a more thumbs up ( likes) by a user indicates user is happy with song recommendations.

In [ ]:
# filter out rows with userId == null
labeled_df = labeled_df.where(psqf.col('user')!='null')
labeled_df.show(5)

In [69]:
thumbsup_count = labeled_df.where((psqf.col('page')=='Thumbs Up') &(psqf.col('userId')!='null')).groupby("userId").agg(psqf.count(psqf.col('page')).alias('thumbsUpCount')).orderBy('userId')
thumbsup_count.show(5, False)

+------+-------------+
|userId|thumbsUpCount|
+------+-------------+
|10    |37           |
|100   |148          |
|100001|8            |
|100002|5            |
|100003|3            |
+------+-------------+
only showing top 5 rows



In [70]:
thumbsdown_count = labeled_df.where((psqf.col("page")=='Thumbs Down')&(psqf.col('userId')!='null')).groupby("userId").agg(psqf.count(psqf.col('page')).alias('thumbsDownCount')).orderBy('userId')
thumbsdown_count.show(5)

+------+---------------+
|userId|thumbsDownCount|
+------+---------------+
|    10|              4|
|   100|             27|
|100001|              2|
|100004|             11|
|100005|              3|
+------+---------------+
only showing top 5 rows



In [75]:
## Join all the features
features_df = churn_event.join(songsplayed, "userId").\
join(hour_count_df, "userId").join(thumbsup_count, "userId").join(thumbsdown_count, "userId")

In [76]:
features_df.show(5)

+------+-------+-----------+---------+-------------+---------------+
|userId|churned|SongsPlayed|HourCount|thumbsUpCount|thumbsDownCount|
+------+-------+-----------+---------+-------------+---------------+
|100010|      0|        275|       26|           17|              5|
|200002|      0|        387|       32|           21|              6|
|   124|      0|       4079|      306|          171|             41|
|    51|      1|       2111|      156|          100|             21|
|     7|      0|        150|       18|            7|              1|
+------+-------+-----------+---------+-------------+---------------+
only showing top 5 rows



In [82]:
assembler = VectorAssembler(inputCols=["SongsPlayed", "HourCount", "thumbsUpCount", "thumbsDownCount"], outputCol="features")
features_df = assembler.transform(features_df)
features_df.select('churned', 'features').show(4)

+-------+--------------------+
|churned|            features|
+-------+--------------------+
|      0|[275.0,26.0,17.0,...|
|      0|[387.0,32.0,21.0,...|
|      0|[4079.0,306.0,171...|
|      1|[2111.0,156.0,100...|
+-------+--------------------+
only showing top 4 rows



In [ ]:
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True)
scalerModel = scaler.fit(features_df)
features_df = scalerModel.transform(features_df)

In [85]:
features_df.select('churned', 'scaledFeatures').show(5, False)

+-------+-------------------------------------------------------------------------------+
|churned|scaledFeatures                                                                 |
+-------+-------------------------------------------------------------------------------+
|0      |[0.2463999205714785,0.2874672791008265,0.2554360395763283,0.3786242371997744]  |
|0      |[0.34675188822240793,0.353805881970248,0.3155386371236997,0.45434908463972923] |
|0      |[3.654782821858403,3.3832687463404962,2.569386045150126,3.10471874503815]      |
|1      |[1.891455390277786,1.7248036746049589,1.5025649386842843,1.5902217962390524]   |
|0      |[0.1343999566753519,0.19901580860826448,0.1051795457078999,0.07572484743995488]|
+-------+-------------------------------------------------------------------------------+
only showing top 5 rows



In [86]:
input_data = features_df.select('scaledFeatures', 'churned')
input_data.show(2)

+--------------------+-------+
|      scaledFeatures|churned|
+--------------------+-------+
|[0.24639992057147...|      0|
|[0.34675188822240...|      0|
+--------------------+-------+
only showing top 2 rows



# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

### Labeled Points
MLlib requires that our features be expressed with LabeledPoints. The required format for a labeled point is a tuple of the response value and a vector of predictors. We can call 'map' on df in order to return an RDD of LabeledPoints.

In [95]:
input_rdd = input_data.rdd.map(lambda line:LabeledPoint(line[-1],[line[:-1]]))
input_rdd.take(5)

[LabeledPoint(0.0, [0.246399920571,0.287467279101,0.255436039576,0.3786242372]),
 LabeledPoint(0.0, [0.346751888222,0.35380588197,0.315538637124,0.45434908464]),
 LabeledPoint(0.0, [3.65478282186,3.38326874634,2.56938604515,3.10471874504]),
 LabeledPoint(1.0, [1.89145539028,1.7248036746,1.50256493868,1.59022179624]),
 LabeledPoint(0.0, [0.134399956675,0.199015808608,0.105179545708,0.07572484744])]

### split datest into train, validation and test sets
- train model using training dataset
- tune model using validation dataset
- test best model selected based on validation dataset on test dataset.

In [98]:
# Split the data into training and test sets (since dataset is imbalanced)
(trainingData, tempData) = input_rdd.randomSplit([0.6, 0.4])
(validationData, testData) = tempData.randomSplit([0.5, 0.5])

In [99]:
model = RandomForest.trainClassifier(trainingData, numClasses=2, categoricalFeaturesInfo={},
                                     numTrees=3, featureSubsetStrategy="auto",
                                     impurity='gini', maxDepth=4, maxBins=32)

In [100]:
# Evaluate model on test instances and compute test error
predictions = model.predict(validationData.map(lambda x: x.features))
labelsAndPredictions = validationData.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(
    lambda lp: lp[0] != lp[1]).count() / float(testData.count())
print('Test Error = ' + str(testErr))
print('Learned classification forest model:')
print(model.toDebugString())

Test Error = 0.1875
Learned classification forest model:
TreeEnsembleModel classifier with 3 trees

  Tree 0:
    If (feature 1 <= 0.07186681977520662)
     Predict: 1.0
    Else (feature 1 > 0.07186681977520662)
     If (feature 3 <= 0.34076181347979695)
      If (feature 2 <= 0.30802581243027827)
       If (feature 0 <= 0.23743992345978837)
        Predict: 0.0
       Else (feature 0 > 0.23743992345978837)
        Predict: 1.0
      Else (feature 2 > 0.30802581243027827)
       Predict: 0.0
     Else (feature 3 > 0.34076181347979695)
      If (feature 1 <= 0.9342686570776861)
       If (feature 0 <= 0.6003198064832386)
        Predict: 1.0
       Else (feature 0 > 0.6003198064832386)
        Predict: 0.0
      Else (feature 1 > 0.9342686570776861)
       Predict: 0.0
  Tree 1:
    If (feature 0 <= 0.057343981514816816)
     Predict: 1.0
    Else (feature 0 > 0.057343981514816816)
     If (feature 2 <= 2.0960780894645765)
      If (feature 3 <= 1.7416714911189621)
       If (feature 3

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.